<h1>0. Import library 🥱</h1>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

<h1>1. Create preprocess class 🥴</h1>

In [ ]:
class preprocess:
    def __init__(self, data_path): # constructor
        self.data_path = data_path

    def chunk_process(self, chunk): # Xử lý dữ liệu trong một chunk
        chunk = chunk[chunk['gender'].isin([0, 1, 2])] # Lọc lấy các user có gender là 0, 1 hoặc 2

        chunk = chunk[['id', 'course_order', 'enroll_time']] # Giữ lấy các cột id, course_order, enroll_time

        chunk = chunk.rename(columns = {'id': 'user', 'course_order': 'course'}) # Đổi tên id thành user, course_order thành course

        chunk = chunk.explode(['course', 'enroll_time']).reset_index(drop=True) # Tạo quan hệ giữa 1 user và 1 course bằng cách tách tập course

        chunk['course'] = chunk['course'].apply(lambda x: f'C_{str(x).replace(" ", "")}') # Format dữ liệu trong cột course

        return chunk

    def load_data(self):
        df = pd.read_json(self.data_path,
                          lines = True,
                          chunksize = 10000) # Load dữ liệu user theo từng chunk 10000 row

        processed_df = [] # List lưu các chunk

        for chunk in df: # Lặp qua từng chunk trong df để xử lý
            chunk = self.chunk_process(chunk)
            processed_df.append(chunk) # Thêm chunk được xử lý vào list

        self.df = pd.concat(processed_df) # Gom các chunk lại thành dataframe

    def filter_with_course(self, course_path):
        course_df = pd.read_json(course_path,
                                 lines = True) # Load dữ liệu course

        self.df = self.df[self.df['course'].isin(course_df['id'].unique())] # Lọc các khóa học hợp lệ trong course

    def get_df(self):
        return self.df

    def to_csv(self, path):
        self.df.to_csv(path, index = False) # Lưu dataframe

<h1>2. Using class 😳</h1>

In [ ]:
data_path = '/content/drive/MyDrive/Nhom12/ĐAMH/Project/Data/raw/entities/user.json'

pre = preprocess(data_path)
pre.load_data()
pre.filter_with_course('/content/drive/MyDrive/Nhom12/ĐAMH/Project/Data/data_prepared/entities/course_merge_teachers_schools.json')
df = pre.get_df()

In [ ]:
pre.to_csv('/content/drive/MyDrive/Nhom12/ĐAMH/Project/Data/transformed_data/user_course.csv')